# Generate shape coefficients

The following steps are performed to obtain all the artifacts for force coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Instantiate path manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CfPathManager
import pathlib

path_manager = CfPathManager(output_path=pathlib.Path("./output/pressure"))

Read parameters file

In [2]:
from cfdmod.use_cases.pressure.force.Cf_config import CfConfig

config_path=pathlib.Path("./fixtures/tests/pressure/Cf_params.yaml")
post_proc_cfg = CfConfig.from_file(config_path)

post_proc_cfg

{'measurement_1': CfConfig(bodies={'building': BodyConfig(surfaces=[], sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[0.0, 10.0, 20.0])), 'building2': BodyConfig(surfaces=['p1_xp', 'p2_xp', 'p3_ym', 'p4_ym', 'p5_ym', 'p6c_yp', 'p7_xm', 'p8_yp', 'p9_yp', 't1_ym', 't2_yp'], sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[0.0, 10.0, 20.0]))}, variables=['Cfx', 'Cfy', 'Cfz'], statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']),
 'measurement_2': CfConfig(bodies={'marquise': BodyConfig(surfaces=['m1_yp', 'm2_zp', 'm3_zm'], sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf])), 'lanternim': BodyConfig(surfaces=['L1_xp', 'L2_yp', 'L3_zp_yp', 'L4_zp_ym', 'L5_ym', 'L6_xm'], sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf]))}, variables=['Cfx', 'Cfy', 'Cfz'], statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurt

Read Mesh from file

In [3]:
from nassu.lnas import LagrangianFormat

mesh_path=pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LagrangianFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read pressure coefficient data

In [4]:
import pandas as pd

cp_data_path=pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")
cp_data = pd.read_hdf(cp_data_path)

cp_data_to_use = (
    cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data
)
cp_data_to_use

,time_step,point_idx,cp
0,10000.0,0,0.268273
1,10000.0,1,0.273546
2,10000.0,2,0.265319
3,10000.0,3,0.276913
4,10000.0,4,0.272593
...,...,...,...
294410,10100.0,2910,0.485344
294411,10100.0,2911,0.474767
294412,10100.0,2912,0.450817
294413,10100.0,2913,0.537118


Add areas to pressure coefficient data

In [5]:
vec_areas = mesh.geometry._cross_prod() / 2
areas_df = pd.DataFrame({"Ax": vec_areas[:, 0], "Ay": vec_areas[:, 1], "Az": vec_areas[:, 2]})
areas_df["point_idx"] = areas_df.index

cp_data = pd.merge(cp_data, areas_df,
    on="point_idx",
    how="left"
)

In [6]:
from cfdmod.use_cases.pressure.force.Cf_data import get_representative_areas, get_geometry_from_mesh
from cfdmod.api.vtk.write_vtk import create_polydata_for_cell_data, write_polydata
from cfdmod.use_cases.pressure.zoning.processing import (
    calculate_statistics,
    combine_stats_data_with_mesh,
    get_indexing_mask,
)

for cfg_label, cfg in post_proc_cfg.items():
    for body_label, body_cfg in cfg.bodies.items():
        body_geom, geometry_idx = get_geometry_from_mesh(body_cfg=body_cfg, mesh=mesh)

        zoning_to_use = body_cfg.sub_bodies.offset_limits(0.1)
        df_regions = zoning_to_use.get_regions_df()
        
        sub_body_idx_array = get_indexing_mask(body_geom, df_regions)
        body_cp = cp_data[cp_data["point_idx"].isin(sub_body_idx_array)].copy()
        
        sub_body_idx = pd.DataFrame({"point_idx": geometry_idx, "region_idx": sub_body_idx_array})
        
        body_data = cp_data[cp_data["point_idx"].isin(geometry_idx)].copy()
        body_data = pd.merge(body_data, sub_body_idx,
            on="point_idx",
            how="left"
        )
        
        body_data["fx"] = body_data["cp"] * body_data["Ax"]
        body_data["fy"] = body_data["cp"] * body_data["Ay"]
        body_data["fz"] = body_data["cp"] * body_data["Az"]
        
        body_cf = (
            body_data.groupby(["region_idx", "time_step"])  # type: ignore
            .agg(
                Fx=pd.NamedAgg(column="fx", aggfunc="sum"),
                Fy=pd.NamedAgg(column="fy", aggfunc="sum"),
                Fz=pd.NamedAgg(column="fz", aggfunc="sum"),
            )
            .reset_index()
        )
        
        Ax, Ay, Az = get_representative_areas(body_geom)
        
        body_cf["Cfx"] = body_cf["Fx"] / Ax
        body_cf["Cfy"] = body_cf["Fy"] / Ay
        body_cf["Cfz"] = body_cf["Fz"] / Az
        
        body_cf_stats = calculate_statistics(body_cf, cfg.statistics, variables=cfg.variables)
        
        body_data_df = combine_stats_data_with_mesh(mesh=body_geom, region_idx_array=sub_body_idx_array, data_stats=body_cf_stats)
        
        polydata = create_polydata_for_cell_data(body_data_df, body_geom)
        write_polydata(path_manager.get_vtp_path(body_label=body_label, cfg_label=cfg_label), polydata)
        
        print(body_cf_stats)

   region_idx   Cfx_avg   Cfx_min   Cfx_max   Cfx_rms  Cfx_skewness  \
0           0  0.044884  0.002140  0.100371  0.028635      0.278742   
1           1  0.043229  0.016525  0.078661  0.017989      0.333512   

   Cfx_kurtosis   Cfy_avg   Cfy_min   Cfy_max   Cfy_rms  Cfy_skewness  \
0     -1.078723 -0.037547 -0.044154 -0.034356  0.002883     -0.681069   
1     -1.031225 -0.083693 -0.092243 -0.067217  0.008043      0.574987   

   Cfy_kurtosis   Cfz_avg   Cfz_min   Cfz_max   Cfz_rms  Cfz_skewness  \
0     -0.799677 -0.029644 -0.034369 -0.019635  0.004647      0.682328   
1     -1.063774  0.481142  0.304503  0.582544  0.087437     -0.488848   

   Cfz_kurtosis  
0     -0.890765  
1     -1.085833  
   region_idx   Cfx_avg   Cfx_min   Cfx_max   Cfx_rms  Cfx_skewness  \
0           0  0.044884  0.002140  0.100371  0.028635      0.278742   
1           1  0.043252  0.016639  0.078498  0.017911      0.331234   

   Cfx_kurtosis   Cfy_avg   Cfy_min   Cfy_max   Cfy_rms  Cfy_skewness  \
0    